<a href="https://colab.research.google.com/github/calcoach/covidStatistics/blob/master/Statistics_covid2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sodapy

In [ ]:

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
dataset = "gt2j-8ykr"
count = client.get(dataset,select ="COUNT(*)")
count = count.pop()
limite = index['COUNT']

results = client.get(dataset, limit = limite)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df.count()

id_de_caso               109505
fecha_de_notificaci_n    109505
c_digo_divipola          109505
ciudad_de_ubicaci_n      109505
departamento             109505
atenci_n                 109505
edad                     109505
sexo                     109505
tipo                     109505
estado                   109505
pa_s_de_procedencia         935
fis                      109505
fecha_diagnostico        109505
fecha_recuperado          45334
fecha_reporte_web        109505
tipo_recuperaci_n         45334
codigo_departamento      109505
codigo_pais                 872
pertenencia_etnica        80711
fecha_de_muerte            3864
nombre_grupo_etnico        1439
dtype: int64

In [ ]:
 #condition = results_df[results_df['estado'] == "Leve"]
 #print(condition.count)

 print("Leves: "+str((results_df.estado == 'Leve').sum()))
 print("Asintomatico: "+str((results_df.estado == 'Asintomático').sum()))
 print("Leves: "+str((results_df.estado == 'Leve').sum()))


 print('\n')


 print("Norte de Santander: "+str((results_df.departamento =='Norte de Santander').sum()))
 print("Barranquilla: "+str((results_df.departamento =='Barranquilla D.E.').sum()))
 print("Atlantico: "+str((results_df.departamento == "Atlantico").sum()))
 



Leves: 85487
Asintomatico: 12437


Norte de Santander: 354
Barranquilla: 14097
Atlantico: 0
